In [ ]:
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
#statement="SELECT count(*) FROM {} ORDER BY dow ASC, time ASC".format(table)
statement="SELECT * FROM {} ORDER BY dow ASC, time ASC".format(table)
#statement="SELECT * FROM {} WHERE dow=2 ORDER BY dow ASC, time ASC".format(table)
cursor=con.execute(statement)
rows = cursor.fetchall()
for row in rows:
    print(row)    
con.close()

In [ ]:
#switch 2 songs
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
song1="shuffle_exclude/Fleetwood_Mac_-_The_Chain.mp3"
song2="shuffle_exclude/1._The_Hand_That_Rocks_The_Cradle_Demo_-_The_Smiths.mp3"
###
#todo create these queries 20190813
day,time=[],[]
for song in song1,song2:
    #print(song)
    statement="SELECT * from {} WHERE name='{}'".format(table,song)
    #con.execute(statement)
    cursor=con.execute(statement)
    rows = cursor.fetchall()
    for row in rows:
        #print(row[:2])
        day.append(row[0])
        time.append(row[1])
#print(day,time)
#output
statement1 ='UPDATE {} SET name="{}" WHERE dow={} AND time="{}"'.format(table,song2,day[0],time[0])
con.execute(statement1)
print(statement1)
statement2 ='UPDATE {} SET name="{}" WHERE dow={} AND time="{}"'.format(table,song1,day[1],time[1])
print(statement2)
con.execute(statement2)
#####
con.commit()
con.close()

In [ ]:
#add song
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
#'shuffle_exclude/Love_Like_a_Sunset_Full_by_Phoenix.mp3',
song,dow,tm='shuffle_exclude/Ryan_Adams_-_I_Wish_You_Would_from_1989_Audio.mp3',2,'02:00:00'
statement ='INSERT INTO {} (name,dow,time) VALUES (\"{}\",{},\"{}\")'.format(table,song,dow,tm)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#remove song
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
# (5, '15:00:00', 'linkin_park-Shadow_Of_The_Day.mp3)
# (2, '10:52:30', shuffle_exclude/Iron_Wine_-_Naked_as_We_Came_OFFICIAL_VIDEO.mp3 swap this out 20190702
dow,tm=4,'23:15:00'
statement ='DELETE FROM {} WHERE dow={} AND time="{}"'.format(table,dow,tm)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#update song
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
#"01 Sunrise.mp3",
#song,dow,tm='linkin_park-Shadow_Of_The_Day.mp3',2,'16:00:00'
statement ='UPDATE {} SET name="shuffle_exclude/Ryan_Adams_-_Out_Of_The_Woods_from_1989_Audio.mp3" WHERE dow=0 AND time="23:07:30"'.format(table)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#find shuffle_exclude songs not on schedule yet
import sqlite3
import operator
import glob
pathMusic='/home/pi/Music/shuffle_exclude/'
filename_segment=pathMusic.count("/")
music_source="*.mp3"
music_list=glob.glob(pathMusic+music_source)
#print(music_list)
for song in music_list:
    #print(song)
    song=song.split("/")
    #print(song[filename_segment])
    song="shuffle_exclude/"+str(song[filename_segment])
    #check db for count of previous plays
    con = sqlite3.connect('/home/pi/.jupyter/soundtrack/soundtrack.db')
    table='song_schedule'
    statement ='SELECT * FROM {} WHERE name="{}" LIMIT 0,1'.format(table,song)
    cursor=con.execute(statement)
    rows = cursor.fetchall()
    #print(len(rows))
    if len(rows) ==0:
        print(song)
        #print(rows)       
con.close()        

In [ ]:
#analytics
import matplotlib.pyplot as plt
import pickle
filename='music_dict.p'
pickle=pickle.load(open(filename,"rb"))
music_dict=pickle
#print(music_dict)
song_durations=[]
for song in music_dict:
    b=song[1][2]
    #print(b)
    song_durations.append(b)
#print(song_durations)    
plt.plot(sorted(song_durations))
plt.show()

In [ ]:
#Tool
import pickle
from soundtrack import *
import sqlite3

filename='music_dict.p'
pickle=pickle.load(open(filename,"rb"))
music_dict=pickle
con=sqlite3.connect('soundtrack.db')
#print(music_dict)
for song in music_dict:
    b=song[1][2]
    print(b) 
#    statement='SELECT * FROM shuffle_count WHERE name="{}"'.format(song[0])
    statement='UPDATE shuffle_count SET duration_sec={} WHERE name="{}"'.format(b,song[0])  
    con.execute(statement)                                                                                                               
    #    rows = cursor.fetchall()                                                                                                                    
#    for row in rows:                                                                                                                            
#        print(row)                                                                                                                            
con.commit()
con.close() 

In [ ]:
#Tool2
import sqlite3

con=sqlite3.connect('soundtrack.db')
statement='SELECT * FROM shuffle_count' 
cursor=con.execute(statement)                                                                                                               
rows = cursor.fetchall()                                                                                                                    
for row in rows:                                                                                                                                
    print(row)                                                                                                                            
con.close() 

In [ ]:
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule2'
#query='DROP TABLE {}'.format(table)
query = 'CREATE TABLE {} (dow INTEGER NOT NULL,time TEXT NOT NULL,name TEXT NOT NULL, PRIMARY KEY (dow,time))'.format(table)
con.execute(query)
con.commit()
con.close()

In [ ]:
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
#query='DROP TABLE {}'.format(table)
query = 'CREATE TABLE {} (dow INTEGER NOT NULL,time TEXT NOT NULL,name TEXT NOT NULL)'.format(table)
con.execute(query)
con.commit()
con.close()

In [ ]:
#to update table
import sqlite3
con = sqlite3.connect('soundtrack.db')
t='song_schedule'
t2='song_schedule2'
statement='INSERT INTO {} SELECT * FROM {}'.format(t2,t)
con.execute(statement)
con.commit()
con.close()

In [11]:
#purge old songs from shuffle_count
from soundtrack import *
mode=getShuffleCountMode()
#print(mode)
import sqlite3

con = sqlite3.connect('soundtrack.db')
table='shuffle_count'
statement ="DELETE FROM {} WHERE cnt<'{}'".format(table,mode-4)
con.execute(statement)
con.commit()
con.close()

19


In [ ]:
#check schedule
#todo: 20190530 find the three closest scheduled songs to now, replace one with a new song, 
#and move the original to nearby time slot
import sqlite3
from datetime import datetime, timedelta
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
current_dow=datetime.now().strftime('%w')
current_time=datetime.now().strftime('%H:%M:%S')
end_tm=datetime.now() + timedelta(0,float(3000))
end_tm=end_tm.strftime('%H:%M:%S')
beg_tm=datetime.now() + timedelta(0,float(-3000))
beg_tm=beg_tm.strftime('%H:%M:%S')
print(current_dow,current_time)
print("Ends @ {}".format(end_tm))
statement="SELECT * FROM {} WHERE dow={} ORDER BY dow ASC, time ASC".format(table,current_dow)
cursor=con.execute(statement)
rows = cursor.fetchall()
for row in rows:
    if str(beg_tm) < row[1] < str(end_tm):
        print(row[1:3])    
con.close()

In [ ]:
import sqlite3 
from datetime import datetime

#test and implement this function below when you get a chance 20190709
def getShuffleCountMode():
    
    con = sqlite3.connect('soundtrack.db')
    table='shuffle_count'
    statement ='SELECT cnt,COUNT(name) shuffle_cnt FROM {} GROUP BY cnt ORDER BY shuffle_cnt ASC'.format(table)
    cursor=con.execute(statement)
    rows = cursor.fetchall()
    for row in rows:
        mode=row[0]
    con.close()
    return mode

mode=getShuffleCountMode()
print(mode)

In [12]:
import sqlite3

con = sqlite3.connect('soundtrack.db')
table='shuffle_count'
statement ='SELECT * FROM {} ORDER BY cnt ASC'.format(table)
cursor=con.execute(statement)
rows = cursor.fetchall()
for row in rows:
    print(row)
con.close()

('/home/pi/Music/Pete_Yorn-The Man.mp3', 21, '2019-07-25 08:33:56', 204)
('/home/pi/Music/the_c002.mp3', 21, '2019-07-24 22:15:48', 219)
('/home/pi/Music/The_Clash-Police_on_my_back.mp3', 21, '2019-07-24 22:41:28', 199)
('/home/pi/Music/elliott_smith-Pitseleh.mp3', 21, '2019-08-12 11:49:03', 204)
('/home/pi/Music/Pete_Yorn-Undercover.mp3', 21, '2019-08-09 17:03:47', 236)
('/home/pi/Music/RoxyMusic-Avalon .mp3', 21, '2019-07-25 22:52:52', 257)
('/home/pi/Music/Gin_Blossoms-Hey_Jealousy-Acoustic.mp3', 21, '2019-07-25 05:05:21', 272)
('/home/pi/Music/ATribeCalledQuest-Scenario.mp3', 21, '2019-07-25 01:37:41', 264)
('/home/pi/Music/sade-no_ordinary_love.mp3', 21, '2019-08-13 17:48:34', 440)
('/home/pi/Music/Bob_Mould_-_See_A_Little_Light.mp3', 21, '2019-07-30 06:26:19', 210)
('/home/pi/Music/PostalService-Track05.mp3', 21, '2019-07-23 12:39:09', 272)
('/home/pi/Music/brick000.mp3', 21, '2019-07-24 23:48:32', 299)
('/home/pi/Music/davide_bowie-Queen_Bitch.mp3', 21, '2019-07-25 20:49:30', 19